In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

# An example of a Low Liquidity Trade

In [140]:
import requests
import s3fs
import pandas as pd

from datetime import timedelta, datetime
import pendulum

import yfinance as yf

from prefect import Client, Task, Flow, task, Parameter
from prefect.schedules import IntervalSchedule
from prefect.environments.storage import Docker

# Homemade
from extractMOCData import CONFIG as cfg
from extractMOCData.moc_data import TsxMocData

from addFeatures.daily import DailyData

from mocIO.read_moc import get_moc_file_lst
#from extractMOCData import read_moc 

In [141]:
# tsxMocData = TsxMocData()

# moc_df = tsxMocData.scrape_moc_data()

## Load Raw Data

In [142]:
moc_file_lst = get_moc_file_lst.run("tsx-moc")

In [143]:
data_flpth = f"s3://{moc_file_lst[-1]}"

In [144]:
moc_df = pd.read_csv(data_flpth, parse_dates=["moc_date"], na_filter=False)
moc_df["Symbol"].fillna("NA", inplace=True)

In [145]:
moc_df[moc_df["Symbol"].isna()==True]

,Symbol,Imbalance Side,Imbalance Size,Imbalance Reference Price,moc_date


## Add daily features
(from yahoo)

In [146]:
# Create price data _list
symbol_clmn="Symbol"
dailyData =  DailyData(symbol_clmn=symbol_clmn)
moc_key_df = dailyData.run(moc_df)

BNS.PR.E
BNS.PR.G
BNS.PR.H
BNS.PR.I
BNS.PR.Z


In [181]:
moc_key_df.columns

Index(['Symbol', 'Imbalance Side', 'Imbalance Size',
       'Imbalance Reference Price', 'moc_date', 'yahoo_symbol'],
      dtype='object')

In [176]:
st_dt = moc_key_df.iloc[0]["moc_date"].strftime('%Y-%m-%d')
st_dt = "2020-04-02"

In [180]:
sym = yf.Ticker("AAV.TO")
sym.history(
    start="2020-04-02",
    end="2020-04-03",
    auto_adjust = True
)
    

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-04-02,1.5,1.7,1.43,1.51,732400,0,0


In [188]:
def get_ohlc(row):
    sym = yf.Ticker(row["yahoo_symbol"])
    
    st_dt = row["moc_date"]
    end_dt = st_dt + timedelta(days=1)
    
    df = sym.history(
            start=st_dt.strftime('%Y-%m-%d'), 
            end=end_dt.strftime('%Y-%m-%d'), 
            auto_adjust=True
        ).head(1)
    
    # Add symbolto ohlc
    df["yahoo_symbol"] = row["yahoo_symbol"]

    return df

ohlc_df_lst = moc_key_df.apply(get_ohlc, axis=1)

In [186]:
ohlc_df_lst[0].rea

,Open,High,Low,Close,Volume,Dividends,Stock Splits,yahoo_symbol
Date,,,,,,,,
2020-04-03,1.6,1.63,1.46,1.61,356400,0,0,AAV.TO


In [169]:
st_dt = moc_key_df.iloc[0]["moc_date"].strftime('%Y-%m-%d')
end_dt = moc_key_df.iloc[0]["moc_date"] # + timedelta(days=1)
st_dt

'2020-04-03'

In [163]:
sym_to_get_lst[:2]

['AAV.TO', 'ABX.TO']

In [172]:
data

Close          High               Low          Open         Volume         
           AAV.TO ABX.TO AAV.TO     ABX.TO AAV.TO ABX.TO AAV.TO ABX.TO  AAV.TO   ABX.TO
Date                                                                                   
2020-04-03   1.61  28.25   1.63  28.809999   1.46   27.9    1.6  27.92  356400  2905900
2020-04-03   1.61  28.25   1.63  28.809999   1.46   27.9    1.6  27.92  356429  2905906

In [ ]:
return sym.history(start=st_dt).head(1)

In [31]:
def get_eod_data(row):
    sym = yf.Ticker(row["yahoo_tsx_symbol"])
    row["currency"] = sym.info["currency"]
    
appiled_df = df[""].apply(get_eod_data, axis=1)
df = pd.concat([moc_key_df, appiled_df], axis='columns')

NameError: name 'df' is not defined

In [92]:
sym =  moc_key_df.iloc[0]["Symbol"]
sym

'AAV'

In [ ]:
pre_client = Client()
project_nm = "MOC"

In [ ]:
try:
    proj_id = pre_client.create_project(project_name=project_nm)
except Exception as err:
    print(err.with_traceback)

In [ ]:
scrape_tsxmoc_fl.storage = Docker(dockerfile="/home/ilivni/MOC/Dockerfile")
scrape_tsxmoc_fl.register(project_name=project_nm)

In [ ]:
Mschedule = IntervalSchedule(
    start_date=datetime.utcnow() + timedelta(seconds=1),
    interval=timedelta(minutes=1),
)
schedule

In [ ]:
moc_df["Symbol"] =moc_df["Symbol"].fillna("NA")

In [ ]:
moc_df[moc_df["Symbol"].isnull()]

In [ ]:
moc_df.to_json(orient="records")